In [23]:
import cv2
import numpy as np
from collections import Counter
import os

print("Imports complete.")


Imports complete.


In [24]:
# ---------------------------
# Fibonacci Code for a Rank
# ---------------------------
def fib_code_for_rank(r):
    """
    Compute the Fibonacci code for an integer 'r' (r >= 1) using a Fibonacci numeral system.
    Expected examples:
      Rank 1 -> "11"
      Rank 2 -> "011"
      Rank 3 -> "0011"
      Rank 4 -> "1011"
      Rank 5 -> "00011"
      Rank 6 -> "10011"
      etc.
    This implementation builds a Fibonacci sequence and creates a code based on a greedy (Zeckendorf) representation.
    """
    if r < 1:
        raise ValueError("Rank must be >= 1")
    
    # Generate Fibonacci numbers (starting with 1, 2, 3, 5, ...)
    fibs = [1, 2]
    while fibs[-1] < r:
        fibs.append(fibs[-1] + fibs[-2])
    
    # Build the Fibonacci representation (greedy algorithm)
    code_bits = []
    remaining = r
    for f in reversed(fibs):
        if remaining >= f:
            code_bits.append('1')
            remaining -= f
        else:
            if code_bits:  # Only add zeros after the first '1'
                code_bits.append('0')
    
    # Reverse the bits so that the code is read from smallest Fibonacci number upward
    code_str = ''.join(code_bits)[::-1]
    
    # Pad with zeros so the length equals the number of Fibonacci numbers used
    code_str = code_str.zfill(len(fibs))
    
    # Append the termination marker "1"
    code_str += "1"
    return code_str

print("Fibonacci coding function defined.")


Fibonacci coding function defined.


In [25]:
# ---------------------------
# Frequency and Code Mapping Functions
# ---------------------------
def get_frequency_table(image):
    """
    Given a single-channel image (numpy array), compute the frequency distribution
    of pixel values. Returns a dictionary {pixel_value: frequency}.
    """
    pixels = image.flatten()
    return dict(Counter(pixels))

def get_code_mapping(freq_table):
    """
    Given a frequency table (dictionary), sort the pixel values by frequency (descending)
    and assign a Fibonacci code based on ranking (starting with rank 1).
    Returns a tuple (code_mapping, sorted_items) where:
      - code_mapping is a dictionary {pixel_value: fibonacci_code}
      - sorted_items is a list of (pixel, frequency) tuples sorted in descending order.
    """
    sorted_items = sorted(freq_table.items(), key=lambda x: (-x[1], x[0]))
    code_mapping = {}
    for rank, (pixel, _) in enumerate(sorted_items, start=1):
        code_mapping[pixel] = fib_code_for_rank(rank)
    return code_mapping, sorted_items

print("Frequency and code mapping functions defined.")


Frequency and code mapping functions defined.


In [26]:
# ---------------------------
# Compression and Decompression Functions
# ---------------------------
def compress_image(image, code_mapping):
    """
    Compress a 2D image (single-channel) by replacing each pixel with its corresponding Fibonacci code.
    Returns the compressed bitstream (a string).
    """
    pixels = image.flatten()
    bitstream = "".join(code_mapping[p] for p in pixels)
    return bitstream

def decompress_image(bitstream, code_mapping, image_shape):
    """
    Decompress a bitstream using the provided code mapping.
    (This function is defined for completeness but will not be used to save any image.)
    Returns the decompressed image (numpy array) with shape image_shape.
    """
    # Create reverse mapping: code -> pixel value
    reverse_mapping = {v: k for k, v in code_mapping.items()}
    decoded_pixels = []
    buffer = ""
    for bit in bitstream:
        buffer += bit
        if buffer in reverse_mapping:
            decoded_pixels.append(reverse_mapping[buffer])
            buffer = ""
    return np.array(decoded_pixels, dtype=np.uint8).reshape(image_shape)

print("Compression and decompression functions defined.")


Compression and decompression functions defined.


In [27]:
# ---------------------------
# Compression Function with Fallback
# ---------------------------
def compress_image(image, code_mapping):
    """
    Compress a 2D image (single-channel) by replacing each pixel with its corresponding Fibonacci code.
    Returns the compressed bitstream (a string).
    """
    pixels = image.flatten()
    bitstream = "".join(code_mapping[p] for p in pixels)
    
    # Calculate original size in bits (8 bits per pixel)
    original_bits = image.size * 8
    # If the compressed bitstream is longer than original, use original 8-bit binary representation.
    if len(bitstream) > original_bits:
        bitstream = "".join(format(p, '08b') for p in pixels)
    return bitstream

# (We define decompress_image for completeness, though it is not used to output an image.)
def decompress_image(bitstream, code_mapping, image_shape):
    """
    Decompress a bitstream using the provided code mapping.
    Returns the decompressed image (numpy array) with shape image_shape.
    """
    reverse_mapping = {v: k for k, v in code_mapping.items()}
    decoded_pixels = []
    buffer = ""
    for bit in bitstream:
        buffer += bit
        if buffer in reverse_mapping:
            decoded_pixels.append(reverse_mapping[buffer])
            buffer = ""
    return np.array(decoded_pixels, dtype=np.uint8).reshape(image_shape)

print("Compression function defined (with fallback to original representation if needed).")


Compression function defined (with fallback to original representation if needed).


In [28]:
# ---------------------------
# Processing Functions for Compression Statistics (No Decompressed Image Saving)
# ---------------------------
def process_single_channel_image(image_path, mode_label):
    """
    Process a single-channel image (for BNW or Grayscale).
    - Loads the image.
    - Computes frequency table.
    - Creates code mapping (Fibonacci codes) based on ranking.
    - Compresses the image into a bitstream (with fallback if needed).
    - Returns compression statistics in a dictionary.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Could not load {image_path}")
        return None
    
    freq_table = get_frequency_table(image)
    code_mapping, sorted_items = get_code_mapping(freq_table)
    bitstream = compress_image(image, code_mapping)
    
    original_bits = image.size * 8  # 8 bits per pixel
    compressed_bits = len(bitstream)
    compression_ratio = compressed_bits / original_bits
    space_saved = 100 * (1 - compression_ratio)
    
    return {
        "Image Name": os.path.basename(image_path),
        "Type": mode_label,
        "Original Size (bits)": original_bits,
        "Compressed Size (bits)": compressed_bits,
        "Compression Ratio": compression_ratio,
        "Space Saved (%)": space_saved
    }

def process_color_image(image_path):
    """
    Process a color image (BGR) by splitting into channels and compressing each channel separately.
    Returns overall compression statistics for the color image.
    """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        print(f"Error: Could not load {image_path}")
        return None
    channels = cv2.split(image)
    overall_original = image.size * 8  # total bits (all channels)
    overall_compressed = 0
    
    # Process each channel
    for channel in channels:
        freq_table = get_frequency_table(channel)
        code_mapping, _ = get_code_mapping(freq_table)
        bitstream = compress_image(channel, code_mapping)
        overall_compressed += len(bitstream)
    
    compression_ratio = overall_compressed / overall_original
    space_saved = 100 * (1 - compression_ratio)
    
    return {
        "Image Name": os.path.basename(image_path),
        "Type": "Color",
        "Original Size (bits)": overall_original,
        "Compressed Size (bits)": overall_compressed,
        "Compression Ratio": compression_ratio,
        "Space Saved (%)": space_saved
    }

print("Processing functions defined.")


Processing functions defined.


In [29]:
# ---------------------------
# Main Processing: Compute Compression Statistics and Display Results Table
# ---------------------------
# Define arrays of image paths (update the paths as needed)
bnw_images = [
    r"C:\Users\user\Desktop\Image Compression\media\bnw200.jpg",
    r"C:\Users\user\Desktop\Image Compression\media\bnw300.jpg"
]

grayscale_images = [
    r"C:\Users\user\Desktop\Image Compression\media\gs50.jpg",
    r"C:\Users\user\Desktop\Image Compression\media\gs100.jpg"
]

color_images = [
    r"C:\Users\user\Desktop\Image Compression\media\color400.jpg",
    r"C:\Users\user\Desktop\Image Compression\media\color500.jpg"
]

# Process images and collect statistics
stats = []
for path in bnw_images:
    res = process_single_channel_image(path, "Black and White")
    if res:
        stats.append(res)
for path in grayscale_images:
    res = process_single_channel_image(path, "Grayscale")
    if res:
        stats.append(res)
for path in color_images:
    res = process_color_image(path)
    if res:
        stats.append(res)

# Print table header
print("{:<20} {:<25} {:>20} {:>25} {:>20} {:>20}".format(
    "Image Name", "Type", "Original Size (bits)", "Compressed Size (bits)",
    "Compression Ratio", "Space Saved (%)"))
print("-" * 150)

# Print each row of statistics
for stat in stats:
    print("{:<20} {:<25} {:>20} {:>25} {:>20.2f} {:>20.2f}".format(
        stat["Image Name"],
        stat["Type"],
        stat["Original Size (bits)"],
        stat["Compressed Size (bits)"],
        stat["Compression Ratio"],
        stat["Space Saved (%)"]
    ))


Image Name           Type                      Original Size (bits)    Compressed Size (bits)    Compression Ratio      Space Saved (%)
------------------------------------------------------------------------------------------------------------------------------------------------------
bnw200.jpg           Black and White                        8388608                   6270069                 0.75                25.25
bnw300.jpg           Black and White                         720000                    432551                 0.60                39.92
gs50.jpg             Grayscale                                20000                     20000                 1.00                 0.00
gs100.jpg            Grayscale                                80000                     80000                 1.00                 0.00
color400.jpg         Color                                  3840000                   3840000                 1.00                 0.00
color500.jpg         Color       